# Overfitting and Regularization with Weight Decay

## Learning from Data: Homework 6

### Questions 2, 3, 4, 5, 6

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt